## Soilmoisture analysis and validation for Mozambique

<div style="line-height: 1.4;">
The aim of this notebook is to show a possible workflow for analyzing and validating soil moisture data in Mozambique retrieved from ground measurements and ASCAT satellite data. Due to the given ground measurement data we focus on 5 different areas Buzi, Chokwé, Mabalane, Mabote and Muanza.
<div>

### Workflow

1. Obtaining ground data (access, visiualize(cardtopy, histogram)... )
2. Obtaining ASCAT satellite data (access, transform data, df wide to lng, visiualize (histogram, spatial image)... )
3. Visual validation (compare histograms)
4. Statistic validation
..

In [ ]:
from datetime import datetime
from importlib import reload
from time import time
import netCDF4 as nc
import ascat.read_native.ragged_array_ts as rat
import numpy as np
import xarray as xr
import geopandas as gpd
import osmnx as ox
import os

In [ ]:
#At this point we want to figure out the bounding box of an area in this case Buzi in Mozambique


# Define location name
place_name = "Buzi, Mozambique"


# Get GeoDataFrame from OpenStreetMap
gdf = ox.geocode_to_gdf(place_name)

# Get bounding box
bbox = gdf.total_bounds  # [minx, miny, maxx, maxy]
print(f"Bounding Box for {place_name}: {bbox}")

I'm not sure if cartopy is really the most elegant way to visualize in this case so I once tried it with cartopy and the other time with folium

In [ ]:
#In order to visualize our bounding box on the map we use the cartopy library


import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import box
import cartopy.io.shapereader as shpreader

# Original Buzi bounding box (small, precise region)
buzi_bbox = [33.8931254, 34.8127126, -20.5695588, -19.582687]  # (lon_min, lon_max, lat_min, lat_max)

# Expanded bounding box (to give context)
lon_min, lon_max = 15, 60  # Wider longitude range
lat_min, lat_max = -7, -40  # Taller latitude range

# Create figure with PlateCarree projection
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.PlateCarree()})

# Set the map extent to the larger bounding box
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Add background map features
ax.add_feature(cfeature.BORDERS, linestyle='-')
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.LAND, facecolor='lightgreen')
ax.add_feature(cfeature.OCEAN, facecolor='lightblue')



# Add gridlines
gl = ax.gridlines(draw_labels=True, linestyle="--", alpha=0.5)
gl.right_labels = False
gl.top_labels = False


# Draw the original Buzi bounding box (in red)
buzi_box = box(buzi_bbox[0], buzi_bbox[2], buzi_bbox[1], buzi_bbox[3])
ax.add_geometries([buzi_box], crs=ccrs.PlateCarree(), facecolor='none', edgecolor='red', linewidth=2, label="Buzi Region")
ax.text(35, -20.4, "Buzi", fontsize=12, color='red', weight='bold',
        transform=ccrs.PlateCarree())

# Add title and legend
plt.title("Mozambique")
plt.legend()

# Show the map
plt.show()


In [ ]:
import folium

# Create map centered on Mozambique
m = folium.Map(location=[-20, 40], zoom_start=5)

# Bounding Box [min_lon, max_lon, min_lat, max_lat]
bb = [34, 35, -21, -20]

# Define the coordinates for the bounding box
coordinates = [
    [bb[2], bb[0]],  # [min_lat, min_lon]
    [bb[2], bb[1]],  # [min_lat, max_lon]
    [bb[3], bb[1]],  # [max_lat, max_lon]
    [bb[3], bb[0]],  # [max_lat, min_lon]
    [bb[2], bb[0]],  # Closing the loop: [min_lat, min_lon]
]

# Create a polygon (bounding box) and add a fill
bounding_box = folium.Polygon(
    locations=coordinates,
    color="#39FF14",  # Border color
    weight=1,
    fill=True,  # Fill the box with color
    fill_color="#39FF14",  # Fill color
    fill_opacity=0.3  # Make the fill semi-transparent
).add_to(m)

# Add a popup to the bounding box (this will be triggered on click inside the filled area)
popup_label = "Buzi"  # Text for the popup
folium.Popup(popup_label).add_to(bounding_box)



# Display the map
m

In [ ]:
#at this point we access the ascat data in form of netCDF files
#Remember: netCDF files are processed products of the ascat (advanced scatterometer) carried on th MetOp satellite 


cell_source = "/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data"
cell_collection = rat.CellFileCollectionStack.from_product_id(cell_source, "H121_V1.0")

In [ ]:
#Here we take a look at the metadata of one netCDF file called "1524.nc"

ncsource= "/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/1524.nc"
ds=nc.Dataset(ncsource)
print(ds)

In [ ]:
#We can access the variables of the netCDF file like a library object
#Be aware not to confuse the metadata from above with the variable keys
#Although the variable keys are already given in the metadata as well (see "variables(dimensions): int64 row_size(locations),...")

print(ds.variables.keys())

In [ ]:
#metadata about how the time data is stored

print(ds.variables['time'])

In [ ]:
#As you can see above (see "units: days since 1970-01-01 00:00:00") we have a big timeframe to look at
#Just to be sure we check when the timeframe starts and ends 
#We also take a look at the time resolution of the first ten timestamps 

# Extract first time value
time_var = ds.variables["time"]  # Access time variable
time_units = time_var.units  # Get the unit string (e.g., "days since 1970-01-01")
time_value_start = time_var[0]  # First timestamp
time_value_10 = time_var[0:9]  # First few timestemps
time_value_end = time_var[-1]   # Last timestamp

# Convert to readable date
first_date = nc.num2date(time_value_start, units=time_units)
last_date = nc.num2date(time_value_end, units=time_units)
first_ten = nc.num2date(time_value_10, units=time_units)


print("First observation date:", first_date, "\n", "Last observation date:", last_date,"\n")
print("First 10 observation dates:") 


for i in range(len(first_ten)):
    print(first_ten[i])

# Close dataset
#ds.close()

In [ ]:
print(ds.variables.keys())

In [ ]:
#We still working with only one netCDF file called "1524.nc"
#Now we gonna take a look at the minimum and maximum latitude and longitude 
lat_var=ds.variables['lat']
lon_var=ds.variables['lon']

print("Max latitude:",max(lat_var),"Min latitude:",min(lat_var),"\n")
print("Max longitude:",max(lon_var),"Min longitude:",min(lon_var))



In [ ]:
#Above we see that the area that is covered by our data is quite big
#Especially compared to our bounding box for Buzi f.e.
#In order to avoid our code to crash due to overload, we have to filter out data and make it smaller
#We do this with the help of the xarray library

In [ ]:
xrds=xr.open_dataset("/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/1524.nc",
                     chunks={"time":100})

xrds

In [ ]:
min_lat=xrds.lat.min().values
max_lat=xrds.lat.max().values
min_lon=xrds.lon.min().values
min_lon=xrds.lon.max().values

print(min_lat)
print(max_lat)
print(min_lon)
print(max_lon)

In [ ]:
#Seemingly we have to filter the NaNs in the xarray 

In [ ]:
#this part might no be needed

# Define location name

area_dict = {"Buzi":{"lat":[],'lon':[]},"Chokwé":{"lat":[],'lon':[]},"Mabalane":{"lat":[],'lon':[]},
             "Mabote":{"lat":[],'lon':[]},"Muanza":{"lat":[],'lon':[]}}

for name in area_dict:
    
    gdf=ox.geocode_to_gdf(name)
    bbox = gdf.total_bounds
    min_lon, min_lat, max_lon, max_lat = bbox
    area_dict[name]["lon"].extend([min_lon, max_lon]) 
    area_dict[name]["lat"].extend([min_lat, max_lat])
    
area_dict


In [ ]:
#this part might no be needed

# Path to the directory
directory_path = "/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/"

# List all files in the directory
files = os.listdir(directory_path)

print(files)

# Loop through the files
for file in files:
    
    ncsource= f"/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/{file}"
    ds=nc.Dataset(ncsource)
    list=ds.variables.keys()
    print(list)
        

In [ ]:
#so at first we tried the bounding box for whole Mozambique but that always crashed
#lets try it with the bounding box just for the area of Buzi

#bounds = (-21,-20,34,35) #latmin, latmax, lonmin, lonmax
bounds = (-27,-10,30,41) #latmin, latmax, lonmin, lonmax

output_bbox = cell_collection.read(bbox=bounds)

#The timeframe we choose for now is a random date until the end of the dataset
#random_date= np.datetime64("2024-05-30 20:07:09.816000")
#dates = (random_date, np.datetime64(last_date))

# Now apply the time range filter
#dates = (random_date, np.datetime64(last_date))
#output_bbox_filtered_by_time = output_bbox.sel(time=slice(dates[0], dates[1]))

#output_bbox_filtered_by_time

#output_bbox = cell_collection.read(bbox=bounds, date_range=dates)
#output_bbox